In [1]:

import numpy as np
import pandas as pd
import psycopg2

from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib 
from sklearn.model_selection import train_test_split
# Importing the xgboost
import catboost
from catboost import CatBoostRegressor,Pool
import matplotlib.pyplot as plt
#import matplotlib.pyplot as plt
import os



In [2]:


# ustawienie plików z parametemai modelu
model_path='D:\polution_prediction\models'
model_name="catboost_polution_model"
model_season='zima'
model_path =model_path+"\\"+model_season
model_file = model_path+'\\'+model_name+".cmb"
csv_path='D:\\polution_prediction\\models\\'+model_season+'\\csv\\'
#scaler_file=model_path+"\\" +model_name+"\\scaler"
#mean_indexbytime_file = model_path+"\\" +model_name+"\\mean_byTime"
#board_log =model_path+"\\" +model_name+"\\graph"

In [3]:
dbuser='global_owner'
dbpass='if(g=l)weatherpg'
try:
    conn = psycopg2.connect(dbname='weatherpg',user=dbuser,host='weatherpg.db.dv.interia.pl',password=dbpass)
except:
    print("I am unable to connect to the database")

In [4]:
# sel_city2_model='''
# select sel3.cit_sta_id,sel3.cit_id,ct.cit_asn_id,ct.cit_name_pl from
# (select sel2.cit_sta_id,weather_owner._wh_bigest_asn_city_id(sel2.cit_sta_id) cit_id from 
# (select  cit_sta_id,count(*) as cnt from weather_owner.wh_cities
# where cit_asn_id is not null 
# group by  cit_sta_id
# order by cnt desc) sel2
# where cnt>=10) sel3
# join  weather_owner.wh_cities ct  on (sel3.cit_id=ct.cit_id)
# '''

sel_city2_model='select cit_sta_id,cit_id,cit_asn_id,cit_name_pl from weather_owner.wh_polution_models'

dataset_cities=pd.read_sql_query(sel_city2_model,con=conn)

In [5]:
col_list=['timeofday','is_weekend','windspeed6','dispel50','temp6','prev_index_avg','opady6','rhumid','wind_dir_ns','wind_dir_ew']
cat_features_index = [0,1]

In [6]:
def build_model(p_sta_id,p_cit_id,p_asn_id,p_name_pl):
    v_sql=f'select * from weather_owner.polution_table_data_season({p_sta_id},{p_asn_id},\'{model_season}\')'
    dataset_raw =pd.read_sql_query(v_sql,con=conn)
    dataset_raw.set_index("data",inplace=True)
    X =pd.DataFrame(dataset_raw, columns=col_list)
    Y =pd.DataFrame(dataset_raw, columns=['index_avg'])
    uniqe_data=pd.unique(dataset_raw.index)
    uniqe_data_train, uniqe_data_test= train_test_split(uniqe_data, test_size = 0.2, random_state = 12)
    X_train=X[X.index.isin(uniqe_data_train)]
    X_test =X[X.index.isin(uniqe_data_test)]
    y_train=Y[Y.index.isin(uniqe_data_train)]
    y_test =Y[Y.index.isin(uniqe_data_test)]
    if y_train.shape[0]<500 :
        return 'za mało danych'
    model = CatBoostRegressor(iterations=7000, learning_rate=0.01, depth=6,l2_leaf_reg=27)
    model.fit(X_train, y_train, cat_features_index,eval_set=[(X_test, y_test)],use_best_model=True,silent=True)
    directory= model_path+f'\\{p_cit_id}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    model_file=directory+'\\'+model_name+".cmb"
    model.save_model(model_file, format="cbm")
    
    score=model.score(X_test, y_test)
    report=f'''
    Model dla {p_name_pl}
    score: {score}
    rozmiar próbki: {X.shape[0]}
    w tym:
    w zbiorze treningowym {X_train.shape} elementów
    w zbiorze testowym {X_test.shape} elementów
    future importance:
    '''
    
    pool = Pool(X_train, y_train)

    fi = model.get_feature_importance(pool,fstr_type='FeatureImportance')
    feature_importance=pd.DataFrame(fi,columns=['importance'])
    feature_importance["futures"]=col_list
    feature_importance.set_index("futures",inplace=True)
    feature_importance.sort_values(by=["importance"],ascending =False,inplace=True )
    fi_report=str(feature_importance.to_string)
    report_file=directory+'\\report.txt'
    f = open(report_file, 'w') 
    
    f.write(report+fi_report)
    f.close()
#     dataset_raw.to_csv(csv_path+f'{p_cit_id}_{p_name_pl}')
    return 'ok'
    

    

In [7]:
#retult=build_model(119,4970,204,'krakow')

In [ ]:
for row in dataset_cities.iterrows():
    cit_sta_id=f'{row[1][0]:.0f}'
    cit_id    =f'{row[1][1]:.0f}'
    cit_asn_id=f'{row[1][2]:.0f}'    
    name_pl   =f'{row[1][3]}'    
    #print(cit_sta_id,cit_id,cit_asn_id)
    result=build_model(cit_sta_id,cit_id,cit_asn_id,name_pl)
    
